# VO2 Max Prediction

This notebook demonstrates the process of predicting VO2 max using a dataset containing various physiological and training-related features.

**Steps:**

1.  **Setup and Imports**: Install necessary libraries and import modules.
2.  **Import and Inspect Dataset**: Load and explore the initial dataset.
3.  **Clean and Prepare Data**: Preprocess the data for analysis.
4.  **Augment with Synthetic Features**: Add simulated features to enrich the dataset.
5.  **Export Augmented Data Set**: Export augmented dataset for use in training pipeline.


### Setup and Imports

In [10]:
!pip -q install numpy pandas scikit-learn streamlit joblib matplotlib

import numpy as np, pandas as pd
import os
from numpy.random import default_rng
rng = default_rng(42)

import os, json, joblib, math

### Import and Inspect Dataset

In [ ]:
!ls -lh /content
df = pd.read_csv("/content/sample_data/SPO2Max dataset/Baseline_Data_Insight1b.csv")
df.head(), df.info(), df.shape

### Clean and Prepare Data

In [12]:
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

# select relevant fields
cols = ["age","sex","height","mass","total_lean_mass","bmi","wbtpf","vo2max_rel"]
df = df[cols].dropna()

# rename for consistency with earlier runbook
df = df.rename(columns={
    "mass": "weight_kg",
    "height": "height_cm",
    "vo2max_rel": "vo2max"
})

# encode sex
if df["sex"].dtype == "object":
    df["sex"] = df["sex"].str.upper().map({"M": 1, "F": 0})

df.head(), df.shape


(    age  sex  height_cm  weight_kg  total_lean_mass        bmi      wbtpf  \
 0  24.2    1      174.0      103.9        64.940172  34.317611  38.403303   
 1  29.6    0      169.0       64.3        45.163956  22.513217  29.592141   
 2  32.9    1      188.4      115.2        79.401800  32.455678  32.061826   
 3  26.7    0      161.1       83.3        47.708081  32.096222  43.168116   
 4  22.5    0      162.8       61.8        45.926730  23.317376  26.905369   
 
    vo2max  
 0    25.3  
 1    36.6  
 2    35.8  
 3    30.4  
 4    56.1  ,
 (424, 8))

# Augment with synthetic features

Since this real data lacks sleep, HR, training, we are simulating those. We will merge these synthetic features on top so the model can train with richer input. For roadmap, we will collect sleep, HR and training data from wearables

In [13]:
N = len(df)
rng = np.random.default_rng(123)

# synthetic HR / recovery
df["resting_hr"] = (rng.normal(60, 6, N) + 0.1*(df["bmi"] - 24)).clip(45, 95)
df["max_hr"] = (220 - df["age"]) + rng.normal(0, 3, N)
df["hr_recovery_1min"] = (30 + rng.normal(0, 4, N) - 0.05*(df["age"] - 30)).clip(10, 55)

# synthetic training load
df["training_hours_week"] = rng.normal(4, 2, N).clip(0, 10)
df["avg_intensity"] = rng.integers(3, 9, N)
df["rest_days"] = rng.integers(0, 3, N)

# synthetic sleep / recovery features
df["sleep_hours_last_night"] = rng.normal(7, 1, N).clip(4, 9)
df["avg_sleep_hours_week"] = (df["sleep_hours_last_night"] + rng.normal(0, 0.7, N)).clip(5, 8)
df["sleep_quality_score"] = (rng.normal(70, 10, N)
                             - 0.3*(df["resting_hr"] - 60)).clip(30, 95)
df["resting_hr_delta"] = df["resting_hr"] - df["resting_hr"].median()

### Export Augmeted Data Set

In [14]:
os.makedirs("assets", exist_ok=True)
df.to_csv("assets/vo2_real_augmented.csv", index=False)
print("Saved → assets/vo2_real_augmented.csv", "rows:", len(df))

Saved → assets/vo2_real_augmented.csv rows: 424
